In [46]:
import os
import pandas as pd
# import tensorflow as tf
import tensorflow

# pip install tf-keras 설치 필요

from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import random
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [53]:
import gc

# 메모리 해제
gc.collect()


0

In [54]:
train_data_path = "./data/train2.csv"
train_data = pd.read_csv(train_data_path, index_col=0)
train_data

# train_data_path ="/aiffel/aiffel/dktc/data/train.csv"
# train_data = pd.read_csv(train_data_path)


,class,conversation
idx,,
1,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
2,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
3,기타 괴롭힘 대화,너 피아노 좀 쳐봐.\n싫어.\n왜 손가락 없다고 유세 떠는 거야?\n이씨.\n비행...
4,갈취 대화,요즘에는 액수가 작네?\n미안해 요즘에 용돈이 작아\n그게 나랑 무슨 상관이야?\n...
5,기타 괴롭힘 대화,야 이거봐 완전 길동이 닯음\n 진짜네 \n야 그러지마.\n왜 똑같구만 원숭이 \n...
...,...,...
4946,직장 내 괴롭힘 대화,어이 신병.\n이병 김범례.\n와봐.\n네.\n네? 뒤질래?\n.자.잘못들었습니다?...
4947,직장 내 괴롭힘 대화,오늘 회의 안건인 길동프로그램의 출연자는 누구로 할 것인가에 대해 모두 의견 내주시...
4948,직장 내 괴롭힘 대화,야 열심히들 해라 새끼들아\n넵 감사해요 부장님\n야 내가 언제 너 한테 말했냐\n...


In [55]:
# 영어, 한국어가 아닌 경우 공백 ( ) 처리
# 두 개 이상의 느낌표(!+)가 있을 경우 느낌표 하나로 처리
# 두 개 이상의 물음표(\?+)가 있을 경우 물음표 하나로 처리
# ?, ., !, , 가 있을 경우 그 주위에 공백을 추가
# 연속적인 공백이 있을 시 공백을 하나로 처리
# 문장 앞뒤의 공백과 개행문자를 제거(strip)


def preprocess_sentence(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r"([^a-zA-Zㄱ-ㅎ가-힣?.!,])", " ", sentence)
    sentence = re.sub(r"!+", "!", sentence)
    sentence = re.sub(r"\?+", "?", sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence


# 영어, 한국어가 아닌 경우 공백 ( ) 처리
# 연속적인 공백이 있을 시 공백을 하나로 처리
# 문장 앞뒤의 공백과 개행문자를 제거(strip)
# 위 함수를 거친 문장은 문법기호(?, !, , 등)도 모두 제거됨
def preprocess_sentence2(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r"([^a-zA-Zㄱ-ㅎ가-힣])", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence


# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data["conversation"], desc="Generate sentences.."):
    # preprocess_sentence2()로 문장(val)을 전처리하여 배열에 저장
    sentences.append(preprocess_sentence2(val))


#### preprocess_sentence() 케이스 사용 시 해당 함수 내용을 써보자
#### 테스트는 preprocess_sentence()를 거친 데이터 기준으로 수행됨
def dummy():
    # 학습할 문장이 담길 배열
    sentences2 = []

    for val in tqdm(train_data["conversation"], desc="Generate sentences.."):
        # preprocess_sentence()로 문장(val)을 전처리하여 배열에 저장
        sentences.append(preprocess_sentence(val))


labels = []

for val in tqdm(train_data["class"], desc="class label convert to num..."):
    if "갈취" in val:
        labels.append(1)
    if "기타" in val:
        labels.append(3)
    if "직장" in val:
        labels.append(2)
    if "협박" in val:
        labels.append(0)
    ######################################################
    if "일반" in val:
        labels.append(4)


# 데이터셋 분할
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)


def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return "".join(new_words)


def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = (
        new_words[random_idx_2],
        new_words[random_idx_1],
    )
    return new_words


def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


class label convert to num...: 100%|██████████| 4950/4950 [00:00<00:00, 1651694.89it/s]


In [56]:
print("before data augmentation: ", len(train_sentences))

train_splted = pd.DataFrame({"sentence": train_sentences, "class": train_labels})
# random deletion
train_splted_rd = train_splted.copy()
train_splted_rd["sentence"] = train_splted_rd["sentence"].apply(random_deletion)

# random swap
train_splted_rs = train_splted.copy()
train_splted_rs["sentence"] = random_swap(train_splted_rs["sentence"])

# with data augmentation
# train_concated = pd.concat([train_splted,train_splted_rd,train_splted_rs])

# without data augmentation
train_concated = pd.concat([train_splted])

print("after data augmentation: ", len(train_concated))

train_concated


before data augmentation:  3960
after data augmentation:  3960


,sentence,class
0,오늘 날씨 어때 맑고 따뜻해 좋네 주말에 계획 있어 등산 갈 생각이야 재밌겠다 최근...,4
1,사장님 배달이 분까지인데 분이나 늦었잖아요 죄송합니다 배달원한테 전달을 했는데 길이...,3
2,이번 달 월급 들어오면 갚는다며 들어오자마자 다 뜯겨서 그래 미안해 그 말이 벌써 ...,1
3,야 우리 내일 현장학습인 거 알지 응 알지 그럼 니가 내 대신 내도시락도 좀 싸와라...,3
4,여기가 어딘가요 야 있는 돈 다 내놔 맞기 싫으면 아니 댁은 누구신대 다짜고짜 이러...,1
...,...,...
3955,야 일 제대로 안해 아니 물량수준이 희망사항 수준인데 그래서 뭐 임마 그 많은 작업...,2
3956,역시 난 니가 올 줄 알았어 우리가족을 인질로 잡아 니가 그러고도 사람이야 당연하지...,0
3957,밤마다 뭘하는 지 시끄러워서 잠을 못자겠다고 밤에 뭐 하긴 자지 자는 데 왜 쿵쾅거...,0
3958,오늘 날씨 어때 맑고 따뜻해 좋네 주말에 계획 있어 등산 갈 생각이야 재밌겠다 최근...,4


In [57]:
# train_sentences = list(train_concated['sentence'].values)
# train_labels = list(train_concated['class'].values)


In [58]:
# BERT 토크나이저 & 모델 준비
# 학습된 BERT 모델 사용 → bert-base-multilingual-cased

# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=5
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
# 토큰 최대 길이
MAX_LEN = 200
# 데이터 묶음 크기
BATCH_SIZE = 32
# Learning Rate
lr = 5e-5  # 5e-5 , 1e-4
# 훈련 횟수
EPOCH = 10


In [60]:
# 데이터셋을 BERT 입력 형식으로 변환
train_encodings = tokenizer(
    train_sentences, truncation=True, padding=True, max_length=MAX_LEN
)  # 뒤쪽에 패딩
val_encodings = tokenizer(
    val_sentences, truncation=True, padding=True, max_length=MAX_LEN
)


In [61]:
# TensorFlow 데이터셋 생성
train_dataset = (
    tensorflow.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
    .shuffle(100)
    .batch(BATCH_SIZE)
)

val_dataset = tensorflow.data.Dataset.from_tensor_slices(
    (dict(val_encodings), val_labels)
).batch(BATCH_SIZE)


In [62]:
# !pip install --upgrade keras


In [69]:
import keras

print(keras.__version__)

# import os
# os.environ["TF_USE_LEGACY_KERAS"] = "1"

opt = tensorflow.compat.v1.train.AdamOptimizer(learning_rate=lr)
# opt = keras.optimizers.Adam(learning_rate=lr)

model.compile(
    optimizer=opt,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


3.3.3


In [72]:
# 스케줄러 설정
reduce_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",  # 모니터링할 지표 (검증 정확도)
    factor=0.5,
    patience=3,
    min_lr=1e-6,
)


epoch = EPOCH

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epoch,
    callbacks=[reduce_lr],
)


AttributeError: 'ReduceLROnPlateau' object has no attribute '_implements_train_batch_hooks'

In [ ]:
import gc

# 메모리 해제
gc.collect()


# 모델 평가
evaluation = model.evaluate(val_dataset)
print("평가 결과:", evaluation)
